In [ ]:
# Librerías generales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Mis propias librerías
import my_data_frequency_conversor as dfc
import my_models as mm
import my_llm_ts_preprocessor as tsp
import my_prompting_tool as mpt 

# Librerías de los métodos tradicionales
from statsmodels.tsa.holtwinters import SimpleExpSmoothing 
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import Holt

# Creación  y carga de archivo .env, para gestionar las API KEYS
from dotenv import load_dotenv
import os
load_dotenv()  # Carga el .env

# Librerías de Nixtla
from nixtla import NixtlaClient

# Librerías de PyTorch
import torch

# Librerías de Moirai
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast
from uni2ts.model.moirai import MoiraiModule

# Librerías de Chronos
from chronos import BaseChronosPipeline


C:\Users\vicde\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#DATA_HOURS = 'DATA/ETHUSD-H1.csv'
#DATA_DAYS = 'DATA/ETHUSD-D1.csv'
DATA_MONTHS = 'DATA/ETHUSD-Monthly.csv'
#arima_h = mm.Model('ARIMA')
#arima_d = mm.Model('ARIMA-d')
arima_m = mm.Model('ARIMA-m')
#exp_smooth_h = mm.Model('EXPONENTIAL SMOOTHING')
#exp_smooth_d = mm.Model('EXPONENTIAL SMOOTHING')
exp_smooth_m = mm.Model('EXPONENTIAL SMOOTHING')
#holt_h = mm.Model('HOLT')
#holt_d = mm.Model('HOLT')
holt_m = mm.Model('HOLT')
#holt_winters_h = mm.Model('HOLT WINTERS')
#holt_winters_d = mm.Model('HOLT WINTERS')
holt_winters_m = mm.Model('HOLT WINTERS')
#timegpt_h = mm.Model('TIMEGPT')
#timegpt_d = mm.Model('TIMEGPT')
timegpt_m = mm.Model('TIMEGPT')
#lagllama_h = mm.Model('LAGLLAMA')
#lagllama_d = mm.Model('LAGLLAMA')
lagllama_m = mm.Model('LAGLLAMA')
#gpt_h = mm.Model('GPT-H')
#gpt_d = mm.Model('GPT-D')
gpt_m = mm.Model('GPT-M')


In [4]:
data_months = pd.read_csv(DATA_MONTHS, delimiter=',')
train_data_months, val_data_months = dfc.split_data(data_months)
train_data_months = train_data_months.asfreq('MS')
val_data_months = val_data_months.asfreq('MS')
print(train_data_months.head())
print(val_data_months.head())

                      y
DateTime               
2021-01-01   736.220000
2021-02-01  1313.868608
2021-03-01  1424.400000
2021-04-01  1919.300000
2021-05-01  2769.400000
                 y
DateTime          
2024-01-01  2286.5
2024-02-01  2283.0
2024-03-01  3339.2
2024-04-01  3650.6
2024-05-01  3020.3


In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

In [ ]:
# Convertir la serie en DataFrame con las columnas correctas
arima_m.nixtla_format = dfc.convert_to_nixtla(train_data_months)
print(arima_m.nixtla_format.head())
arima_m.model = StatsForecast(
    models=[AutoARIMA(season_length=1)],
    freq='MS'
)

arima_m.model.fit(arima_m.nixtla_format)
arima_m.forecast = arima_m.model.predict(h=12, level=[95])
print(arima_m.forecast)

In [ ]:
# Plot ARIMA forecast against the actual data with the confidence intervals
train_df = train_data_months.reset_index().iloc[:, :2]  # Solo tomar las dos primeras columnas
train_df.columns = ['ds', 'y']
train_df['type'] = 'Train'

val_df = val_data_months.reset_index().iloc[:, :2]  # Solo tomar las dos primeras columnas
val_df.columns = ['ds', 'y']  # Renombrar columnas
val_df['type'] = 'Validation'

# Concatenar los datos de entrenamiento y validación para graficar
all_data = pd.concat([train_df, val_df])

# Graficar los datos de entrenamiento y validación
plt.figure(figsize=(12, 6))
plt.plot(all_data['ds'], all_data['y'], label='Datos Reales', color='blue')

# Graficar las predicciones
plt.plot(arima_m.forecast['ds'], arima_m.forecast['AutoARIMA'], label='Predicciones', color='red')

# Graficar el intervalo de confianza
plt.fill_between(
    arima_m.forecast['ds'],
    arima_m.forecast['AutoARIMA-lo-95'],
    arima_m.forecast['AutoARIMA-hi-95'],
    color='pink',
    alpha=0.3,
    label='Intervalo de Confianza 95%'
)

# Añadir leyenda y etiquetas
plt.legend()
plt.title('Predicciones vs Datos Reales')
plt.xlabel('Fecha')
plt.ylabel('Eth(USD)')

# Mostrar la gráfica
plt.show()

In [ ]:
# Ajustar el modelo con cálculo óptimo de alpha
exp_smooth_m.model = SimpleExpSmoothing(train_data_months).fit(optimized=True)

# Obtener el alpha óptimo
exp_smooth_m.optimal_alpha = exp_smooth_m.model.model.params["smoothing_level"]
print(f"Alpha óptimo: {exp_smooth_m.optimal_alpha:.4f}")

# Predicción a futuro (próximo año)
exp_smooth_m.forecast = exp_smooth_m.model.forecast(steps=12)
exp_smooth_m.forecast


In [ ]:
holt_m.model = Holt(train_data_months).fit(optimized=True)
holt_m.optimal_alpha = holt_m.model.model.params["smoothing_level"]
holt_m.optimal_beta = holt_m.model.model.params["smoothing_trend"]
print(f"Alpha óptimo: {holt_m.optimal_alpha:.4f}")
print(f"Beta óptimo: {holt_m.optimal_beta:.4f}")
holt_m.forecast = holt_m.model.forecast(steps=12)
holt_m.forecast

In [ ]:
# Ajustar el modelo con cálculo óptimo de alpha
holt_winters_m.model = ExponentialSmoothing(train_data_months, trend='add', seasonal='add', seasonal_periods=12).fit(optimized=True)

# Obtener el alpha óptimo
holt_winters_m.optimal_params = holt_winters_m.model.params
print(f"Parámetros óptimos: {holt_winters_m.optimal_params}")

# Predicción a futuro (próximo año)
holt_winters_m.forecast = holt_winters_m.model.forecast(steps=12)
holt_winters_m.forecast

In [ ]:
train_data_months_nixtla = dfc.convert_to_nixtla(train_data_months)

In [ ]:
my_api_key = os.getenv("MY_NIXTLA_API_KEY")
nixtla_client = NixtlaClient(
    api_key = my_api_key
)


#nixtla_client.plot(train_data_months_nixtla, timegpt_m.predictions)

'''timegpt_d.predictions = nixtla_client.predict(
    df=train_data_days,
    horizon=366,
    time_col='DateTime',
    target_col='Open',
)

timegpt_h.predictions = nixtla_client.predict(
    df=train_data_hours,
    horizon=366*24,
    time_col='DateTime',
    target_col='Open',
)'''


In [ ]:
SIZE = "small"  # model size: choose from {'small', 'base', 'large'}
PDT = 12  # prediction length: any positive integer
CTX = 12*3  # context length: any positive integer
PSZ = "auto"  # patch size: choose from {"auto", 8, 16, 32, 64, 128}
BSZ = 32  # batch size: any positive integer
TEST = 12  # test set length: any positive integer

# Convert into GluonTS dataset
df = dfc.convert_to_moirai(data_months)

ds = PandasDataset(dict(df))

# Split into train/test set
train, test_template = split(
    ds, offset=-TEST
)  # assign last TEST time steps as test set



# Construct rolling window evaluation
test_data = test_template.generate_instances(
    prediction_length=PDT,  # number of time steps for each prediction
    windows=TEST // PDT,  # number of windows in rolling window evaluation
    distance=PDT,  # number of time steps between each window - distance=PDT for non-overlapping windows
)

# Prepare pre-trained model by downloading model weights from huggingface hub
model = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-{SIZE}"),
    prediction_length=PDT,
    context_length=CTX,
    patch_size=PSZ,
    num_samples=100,
    target_dim=1,
    feat_dynamic_real_dim=ds.num_feat_dynamic_real,
    past_feat_dynamic_real_dim=ds.num_past_feat_dynamic_real,
)

predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data.input)

input_it = iter(test_data.input)
label_it = iter(test_data.label)
forecast_it = iter(forecasts)

inp = next(input_it)
label = next(label_it)
forecast = next(forecast_it)

plot_single(
    inp, 
    label, 
    forecast, 
    context_length=200,
    name="pred",
    show_label=True,
)
plt.show()


In [ ]:
%pip install chronos-forecasting

In [ ]:
# Cargar modelo
pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",  # usa "amazon/chronos-bolt-small" para Chronos-Bolt
    device_map="cpu",
    torch_dtype=torch.bfloat16,
)

# Convertir datos (dfc y data_months deben estar definidos antes)
df = dfc.convert_to_moirai(data_months)

# --- 1. Usar solo datos hasta 2023 para entrenar el contexto ---
df_context = df[df.index < "2024-01-01"]

# Predecir 12 meses (todo 2024)
quantiles, mean = pipeline.predict_quantiles(
    context=torch.tensor(df_context["y"].values, dtype=torch.float32).unsqueeze(0),
    prediction_length=12,
    quantile_levels=[0.1, 0.5, 0.9],
)

# --- 2. Construir índice de forecast para 2024 ---
forecast_index = pd.date_range(start="2024-01-01", periods=12, freq="MS")

low, median, high = quantiles[0, :, 0], quantiles[0, :, 1], quantiles[0, :, 2]

# --- 3. Plotear ---
plt.figure(figsize=(10, 5))

# Datos históricos completos (incluye 2024 real)
plt.plot(df.index, df["y"], color="royalblue", label="historical data")

# Predicción (para 2024)
plt.plot(forecast_index, median, color="tomato", label="median forecast")
plt.fill_between(forecast_index, low, high, color="tomato", alpha=0.3, label="80% prediction interval")

plt.axvline(pd.Timestamp("2024-01-01"), color="gray", linestyle="--", label="forecast start")

plt.legend()
plt.grid()
plt.title("Forecast vs Real Data (2024)")
plt.show()



In [5]:
# Convertir la serie temporal a lenguaje natural
desp = mpt.paraphrase_initial('ETH_MONTHS')
train_lan = mpt.paraphrase_seq2lan(train_data_months, desp)
print(train_lan)
# Realizar la predicción para los próximos 12 meses (1 año)
'''
steps = 12  # Número de meses a predecir
model_name = 'gpt-4o-mini'  
predicted_lan = mpt.paraphrasing_predict_llm(desp, train_lan, steps, model_name)
print(predicted_lan)
# Convertir la predicción en lenguaje natural de vuelta a una serie temporal
predicted_series = mpt.recover_lan2seq(predicted_lan)

# Mostrar la serie temporal predicha
print(predicted_series)
'''

This is a monthly time series dataset describing the price of Ethereum (ETH) in US dollars (USD). Each value represents the average price of Ethereum in USD for that month. from 736.22 increasing to 1313.8686084, from 1313.8686084 increasing to 1424.4, from 1424.4 increasing to 1919.3, from 1919.3 increasing to 2769.4, from 2769.4 decreasing to 2705.1, from 2705.1 decreasing to 2275.5, from 2275.5 increasing to 2530.0, from 2530.0 increasing to 3431.1, from 3431.1 decreasing to 3000.5, from 3000.5 increasing to 4288.3, from 4288.3 increasing to 4632.9, from 4632.9 decreasing to 3677.6, from 3677.6 decreasing to 2689.6, from 2689.6 increasing to 2925.3, from 2925.3 increasing to 3282.0, from 3282.0 decreasing to 2728.1, from 2728.1 decreasing to 1941.3, from 1941.3 decreasing to 1071.2, from 1071.2 increasing to 1677.7, from 1677.7 decreasing to 1554.4842756, from 1554.4842756 decreasing to 1329.2, from 1329.2 increasing to 1573.6, from 1573.6 decreasing to 1295.6, from 1295.6 decreasin

"\nsteps = 12  # Número de meses a predecir\nmodel_name = 'gpt-4o-mini'  \npredicted_lan = mpt.paraphrasing_predict_llm(desp, train_lan, steps, model_name)\nprint(predicted_lan)\n# Convertir la predicción en lenguaje natural de vuelta a una serie temporal\npredicted_series = mpt.recover_lan2seq(predicted_lan)\n\n# Mostrar la serie temporal predicha\nprint(predicted_series)\n"

In [ ]:
from huggingface_hub import login

HF_TOKEN = os.getenv("HF_TOKEN")

login(token=HF_TOKEN)




In [ ]:
import transformers
import torch

model_id = "meta-llama/Llama-3.1-8B"

# Asegúrate de haber iniciado sesión en Hugging Face si el modelo es privado
# huggingface-cli login

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
)

prompt = """
<|begin_of_text|>
Tengo una serie mensual de ventas en el siguiente formato:
Enero 2023: 120
Febrero 2023: 130
Marzo 2023: 150
Abril 2023: 170
Mayo 2023: 160

Predice los valores para Junio, Julio, Agosto, Septiembre, Octubre, Noviembre y Diciembre de 2023.
Responde únicamente con los meses y los valores en el mismo formato.
<|eot_id|>
"""

# Almacena el resultado en una variable
outputs = pipeline(prompt)

# El resultado es una lista de diccionarios, extrae el texto generado
generated_text = outputs[0]['generated_text']

# Imprime solo la parte de la predicción
print(generated_text)

In [ ]:
import transformers
import torch

# Usa un modelo más pequeño y eficiente como Gemma 2B
model_id = "google/gemma-2b-it"

pipeline = transformers.pipeline(
    "text-generation", 
    model=model_id, 
    model_kwargs={"torch_dtype": torch.bfloat16}, 
    device_map="auto"
)

# Nuevo prompt para guiar mejor la respuesta
prompt = """
<|start_of_turn|>user
Tengo una serie mensual de ventas en el siguiente formato:
Enero 2023: 120
Febrero 2023: 130
Marzo 2023: 150
Abril 2023: 170
Mayo 2023: 160

Basado en estos datos, predice los valores de ventas para Junio, Julio y Agosto de 2023.
Responde únicamente con una lista de los meses y sus valores.
<|end_of_turn|>
<|start_of_turn|>model
"""

# Almacena el resultado de la generación
outputs = pipeline(
    prompt,
    max_new_tokens=2000, # Establece el número máximo de tokens a generar
    do_sample=True,
    temperature=0.7,
    pad_token_id=pipeline.tokenizer.eos_token_id
)

# Extrae solo la parte generada del texto
# El resultado es una lista de diccionarios
generated_text = outputs[0]['generated_text']

# Elimina el prompt para ver solo la respuesta del modelo
response = generated_text.replace(prompt, "")

print(response.strip())

In [ ]:
from transformers import pipeline

# Modelo de lenguaje pequeño y gratuito
pipe = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1")

prompt = """
Tengo una serie mensual de ventas:
Enero 2023: 120
Febrero 2023: 130
Marzo 2023: 150
Abril 2023: 170
Mayo 2023: 160

Predice los valores de Junio a Diciembre 2023 en el mismo formato.
"""

response = pipe(prompt, max_new_tokens=100, do_sample=True)
print(response[0]["generated_text"])
